In [6]:
import mnemosyne as ms
from mnemosyne.engines import ReturnsEngine 
from datetime import datetime as Datetime
import polars as pl

symbol_enum = pl.Enum(['BTC', 'ETH', 'BTCDOWN', 'BNB', 'SHIB'])

query_lf = pl.DataFrame({
    'symbol': ['BTC', 'ETH', 'BTCDOWN', 'BNB', 'SHIB'], 
    'times': [Datetime(2025, 8, 1, 9, 0, 0)] * 5
}).with_columns(pl.col('symbol').cast(symbol_enum)).lazy()

backend_db = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDC', 
            grid_interval='10m', 
            dataset_type=ms.DatasetType.BinanceSpotTrades, 
        ).lazyframe()

re = ReturnsEngine(
    backend_db, 
    backend_fair_expr = pl.col('vwap_total_by_base'), 
    backend_time_expr = pl.col('last_trade_time')
)

value = re.query(query_lf, 
    start_time_expr = pl.col('times'), 
    mark_duration=pl.lit('10m'), 
    tick_lag_tolerance=pl.lit('30s')
)
value.collect()

sys:1: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided


symbol,times,max_tick_to_query_lag,return
enum,datetime[μs],duration[μs],f64
"""BTC""",2025-08-01 09:00:00,3s 766592µs,0.001697
"""ETH""",2025-08-01 09:00:00,399614µs,0.002277
"""BTCDOWN""",2025-08-01 09:00:00,null,null
"""BNB""",2025-08-01 09:00:00,1s 88569µs,0.002054
"""SHIB""",2025-08-01 09:00:00,5s 547004µs,0.002032
